In [ ]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

import warnings
warnings.simplefilter(action='ignore')

import sys
import spacy

def main():
    print("Enter text file name: ")
    filename = input().strip()
    filepath = os.path.join(sys.path[0], filename)

    if not os.path.exists(filepath):
        print(f"Error: File '{filename}' not found.")
        sys.exit(1)

    try:
        nlp = spacy.load("en_core_web_sm")
    except:
        print("SpaCy model 'en_core_web_sm' not found. Install it using:")
        print("python -m spacy download en_core_web_sm")
        sys.exit(1)

    with open(filepath, "r", encoding="utf-8") as f:
        content = f.read()

    print("=== Original Text Sample (First 300 chars) ===")
    print(content[:300])
    print()

    doc = nlp(content)

    print("=== Sentences with PERSON as Main Subject ===\n")

    for sent in doc.sents:
        persons = [ent.text for ent in sent.ents if ent.label_ == "PERSON"]
        if not persons:
            continue

        root = [tok for tok in sent if tok.dep_ == "ROOT"]
        if not root:
            continue
        root = root[0]
        if root.lemma_ not in ("be", "take"):
            continue

        subject = None
        for token in sent:
            if token.dep_ in ("nsubj", "nsubjpass") and token.ent_type_ == "PERSON":
                subject = token.text
                break
        if not subject:
            subject = persons[0]

        print(f"Sentence : {sent.text.strip()}")
        print(f"Subject  : {subject}")
        print(f"Main Verb: {root.text}")
        print("-" * 50)

if __name__ == "__main__":
    main()
